In questo colab si utilizzano tecniche Monte Carlo, in particolare il metodo del rigetto, per stimare la dose che prenderebbe un'astronauta per un viaggio di 700 giorni nel sistema solare

In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import statistics

N_runs = 10**5        # Number of simulations, to obtain the correct statistics


N_Avog   = 6.022e23    # Avogadro number
e_charge = 1.60e-19    # electron charge (C)
e_radius = 2.82e-13    # classical electron radius (cm)
e_mass   = 0.511       # electron mass (MeV)
Z_tar    = 10.         # water atomic number
A_tar    = 18.         # water mass number
c        = 3.e10       # speed of light (cm/s)
J        = 1.6e-13     # conversion from MeV to J
rho      = 1.          # water density (g/cm3)
I_pot    = 78./1.e6    # Ionization potential of water = 78 eV converted in MeV

# Parameters of the target (man = sphere)

### Define the radius of the sphere ###
radius =    15                         # cm   (the sphere is centered in the origin)
###
S      = 4*math.pi*radius**2         # cell surface in cm^2
V      = 4./3.*math.pi*radius**3     # volume

# Parameters of the cosmic rays
### Define the flux of cosmic rays ###
flux =       4e5     # ions/(cm^2*day)
###

### Variabili per salvare l'energia depositata a vari spessori dello schermo ###
spessori = []
energie_depositate = []
dose = []

Definisco la funzione Bethe-Bloch

In [2]:
def bethe(Ekin, rho, Z_tar, A_tar, I_pot):
  if Ekin > 0.1:
    gamma    = 1 + Ekin/m_proj            # relativistic gamma
    beta     = math.sqrt(1-1/gamma**2)    # beta = v/c as a function of kinetic energy
    const    = 4.*math.pi*N_Avog*e_radius**2*e_mass*rho*Z_tar/A_tar*z_proj**2       # constant term in bethe-bloch
    factor   = (1./beta**2)*(math.log(2*e_mass*gamma**2*beta**2/I_pot)-beta**2)     # energy-dependent term (I_pot converted from eV to MeV)
    dEdx     = const*factor    # stopping power = bethe-bloch = dE/dx
    if dEdx*dx > Ekin:
      dEdx = Ekin/dx
    dE = dEdx*dx
  else:
    dE = Ekin
    dEdx = dE/dx
#  E_dep    = E_dep + dE      # Total deposited energy
  Ekin        = Ekin - dE
  return dE, Ekin

In [3]:
### Parametri di input per la funzione bethe ###

#Silicio
rho_Si= 2.33       #densità del target g/cm^3
Z_tar_Si=  14    #Z del target
A_tar_Si=   28   #A target
I_pot_Si=  8.15*10**(-6)    #Ionization potential of water in MeV

#Alluminio#
rho_Al= 2.70       #densità del target g/cm^3
Z_tar_Al=  13    #Z del target
A_tar_Al=   26   #A target
I_pot_Al=  5.98*10**(-6)    #Ionization potential of water in MeV

In [4]:
strato_Si = 3. #spessore strato di silicio in cm
E_errore=[]

for strato_Al in range(3,4):      #Lo spessore dell'alluminio viene fatto variare
  random.seed(15)
  E_dep = 0.
  E_dep_Si = 0.
  E_dep_Al=0.
  
  
  for n in range(N_runs):
    E_dep_Si_i= 0.
    E_dep_Al_i= 0.
    E_dep_sfera_i=0.
    
    phi1   = random.random()*2*math.pi
    theta1 = math.acos(1 - 2*random.random())

    x1    = radius*math.sin(theta1)*math.cos(phi1)          # cartesian coordinates of the origin of the ion (on the sphere surface)
    y1    = radius*math.sin(theta1)*math.sin(phi1)
    z1    = radius*math.cos(theta1)

    phi2   = 2*math.pi*random.random()          # phi is uniform between 0 and 2pi
    theta2 = math.acos(1 - 2*random.random())

    r = -(2*x1*math.sin(theta2)*math.cos(phi2)+2*y1*math.sin(theta2)*math.sin(phi2)+2*z1*math.cos(theta2))

    if r > 0.:
      ion_rand = random.random()

      if ion_rand < 0.87:
        ion_type = 1                                  # Proton
        m_proj   = 938.
        z_proj   = 1
      elif ion_rand >=0.87 and ion_rand < 0.99:
        ion_type = 2                                  # He ion
        m_proj   = 938.*4.
        z_proj   = 2
      else:
        ion_type = 3                                  # Carbon ion
        m_proj   = 938.*12.
        z_proj   = 6

      E = random.gauss(1000, 200)  #MeV
      
      if ion_type == 1:
        E = E*1
      if ion_type == 2:
        E = E*4
      if ion_type == 3:
        E = E*12

      x     = 0.
      dx    = 0.01

      while x <= strato_Si:
        x = x + dx
        dE, E = bethe(E, rho_Si, Z_tar_Si, A_tar_Si, I_pot_Si)  #Si
        E_dep_Si = E_dep_Si + dE  
        E_dep_Si_i = E_dep_Si_i + dE
        
      x     = 0.
      dx    = 0.01
      E_1 = max(E-E_dep_Si_i,0)
      while x <= strato_Al:          #nello strato protettivo alluminio

         x = x + dx
         dE, E_1 = bethe(E_1, rho_Al, Z_tar_Al, A_tar_Al, I_pot_Al)  #Si
         E_dep_Al = E_dep_Al + dE    
         E_dep_Al_i = E_dep_Al_i + dE
      
      x     = 0.                #nella sfera d'acqua (astronauta)
      dx    = 0.01
      
      E_2 = max(E_1-E_dep_Al_i,0)      #do in input energie attenuate dallo schermo
      while x <= r:
         x = x + dx
         dE, E_2 = bethe(E_2, rho, Z_tar, A_tar, I_pot )
         E_dep = E_dep + dE
         E_dep_sfera_i=E_dep_sfera_i+dE
      E_errore=np.append(E_errore,E_dep_sfera_i)
      
   
  
  energie_depositate.append(E_dep/N_runs)
  spessori.append(strato_Al) 
  dose.append(E_dep*flux*S*J*700/((V*rho*1.e-3)*N_runs))


In [5]:
print(energie_depositate)
print(dose)

dose_m=0.26546214842818755

[30.843475753195882]
[0.27635754274863505]


In [6]:
Errore=np.std(E_errore)/np.sqrt(N_runs)
Err_rel=Errore/energie_depositate
Err_dose=Err_rel*dose


In [7]:
print("Energia media depositata da un nucleone in MeV= ", energie_depositate)
print('Dose per intera missione in Gray = ', dose)
print('Errore sulla dose in Gray =', Err_dose)
print("Spessore silicio in cm = ", strato_Si)
print("Spessore alluminio in cm = ", strato_Al)

Energia media depositata da un nucleone in MeV=  [30.843475753195882]
Dose per intera missione in Gray =  [0.27635754274863505]
Errore sulla dose in Gray = [0.00497192]
Spessore silicio in cm =  3.0
Spessore alluminio in cm =  3
